# Preparazione

In [1]:
RANDOM_SEED = 42

In [2]:
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=5784d9a6afb78b7b648a3fd14557fb811acc303b7fedc6b6dd5b2c785982ec5d
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [3]:
!ngrok authtoken 2Tn9uykXlYRjLcSLIgmv1iLdEvG_3gqA9M3V4hLGqH9N3JWEr

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [4]:
from pyngrok import ngrok
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=6ce7c6f000bddf629328524bc0ccb139b39c9ecd7c740a8f2e601616be4dcd10
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [6]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql import functions as sqlf
from pyspark import SparkContext, SparkConf
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.mllib.evaluation import RankingMetrics

In [7]:
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '4G').\
                set('spark.driver.memory', '45G').\
                set('spark.driver.maxResultSize', '10G').\
                setAppName("VideogameRecommenderSystem").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [8]:
port = '4050'
public_url = ngrok.connect(port).public_url

In [9]:
print("To access the Spark Web UI console, please click on the following link to the ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

To access the Spark Web UI console, please click on the following link to the ngrok tunnel "https://d158-34-105-119-107.ngrok-free.app" -> "http://127.0.0.1:4050"


In [10]:
spark

In [11]:
# Connect this colab to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# ALS analysis

In [12]:
als_model = ALSModel.load('/content/gdrive/MyDrive/VideogameRecommenderSystem/models/ALS11_08_2023__11_12_50')

In [13]:
user_activity_df = spark.read.load('/content/gdrive/MyDrive/VideogameRecommenderSystem/user_activity',
                           format="csv",
                           sep=",",
                           inferSchema="true",
                           header="true")

In [ ]:
user_activity_df.agg(sqlf.countDistinct("steam_int_id").alias("distinct_id_count")).show()

+-----------------+
|distinct_id_count|
+-----------------+
|           291652|
+-----------------+



In [ ]:
user_activity_df.count()

6779243

In [ ]:
top_20_distinct_values = user_activity_df.groupBy('steam_id').count().orderBy(sqlf.col("count").desc()).limit(20)

In [ ]:
demo_df = user_activity_df.join(top_20_distinct_values, on='steam_id', how="inner")

In [ ]:
demo_df = demo_df.drop("count")

In [ ]:
demo_df.count()

70707

In [ ]:
demo_df.agg(sqlf.countDistinct("steam_id").alias("distinct_id_count")).show()

+-----------------+
|distinct_id_count|
+-----------------+
|               20|
+-----------------+



In [ ]:
demo_df.write.options(header='True', delimiter=',').csv('/content/gdrive/MyDrive/VideogameRecommenderSystem/demo_data')

In [ ]:
user_activity_df.show(n=5)

+-----------------+-----------+---------------+------+------------+
|         steam_id|steam_appid|    recommended|rating|steam_int_id|
+-----------------+-----------+---------------+------+------------+
|76561198259233672|     951650|    Recommended|   3.0|           0|
|76561198341423566|    1581451|    Recommended|   3.0|           1|
|76561198031569409|       2310|    Recommended|   3.5|           2|
|76561198031569409|     337000|    Recommended|   5.0|           2|
|76561198031569409|    1353270|Not Recommended|   2.0|           2|
+-----------------+-----------+---------------+------+------------+
only showing top 5 rows



In [ ]:
(training, test) = user_activity_df.randomSplit([0.8, 0.2], seed=RANDOM_SEED)

In [ ]:
predictions = als_model.transform(test)

In [ ]:
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

In [ ]:
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 1.0652216001618737


In [23]:
steam_int_id = user_activity_df.filter(user_activity_df.steam_id == 76561198116879965).first()['steam_int_id']

In [24]:
steam_int_id

168492

In [25]:
user = spark.createDataFrame([168492], "int").toDF("steam_int_id")

In [26]:
steam_id = 168492
user_recs = als_model.recommendForUserSubset(user, 5)
print(f"Top 5 recommendations for User {steam_id}:")
user_recs.show(truncate=False)

Top 5 recommendations for User 168492:
+------------+-------------------------------------------------------------------------------------------------------------+
|steam_int_id|recommendations                                                                                              |
+------------+-------------------------------------------------------------------------------------------------------------+
|168492      |[{2184270, 4.9305077}, {1511610, 4.8229003}, {1262920, 4.803187}, {1304290, 4.6724615}, {1263100, 4.6717854}]|
+------------+-------------------------------------------------------------------------------------------------------------+



In [ ]:
# Convert the predictions DataFrame to include all predictions per user
# Generate top-k recommendations for each user
userRecs = als_model.recommendForAllUsers(5)  # Top-5 recommendations for each user

# Prepare the input for RankingMetrics
user_ground_truth = test.groupby('steam_int_id').agg(sqlf.collect_list('steam_appid').alias('ground_truth_items'))
user_train_items = training.groupby('steam_int_id').agg(sqlf.collect_list('steam_appid').alias('train_items'))

# Join the recommendations and ground truth data on the user ID
user_eval = userRecs.join(user_ground_truth, on='steam_int_id').join(user_train_items, on='steam_int_id') \
    .select('steam_int_id', 'recommendations.steam_appid', 'ground_truth_items', 'train_items', 'recommendations.rating')
user_eval = user_eval.toPandas()

In [ ]:
user_eval.head()

,steam_int_id,steam_appid,ground_truth_items,train_items,rating
0,3,"[2184270, 1511610, 1262920, 1304290, 1263100]",[410820],"[261570, 296490]","[4.871675491333008, 4.765350818634033, 4.74587..."
1,6,"[2184270, 1511610, 1262920, 1304290, 1263100]",[1929610],"[739630, 771300, 962130, 1304930, 1677740]","[6.077251434326172, 5.944612979888916, 5.92031..."
2,9,"[2184270, 1511610, 1262920, 1304290, 1263100]","[264710, 346110, 794260]","[251570, 354680, 424370, 455120, 728240, 79074...","[5.162646770477295, 5.0499701499938965, 5.0293..."
3,12,"[2184270, 1511610, 1262920, 1304290, 1263100]",[755790],"[221100, 267600, 506540, 677620, 1149460, 1307...","[5.699399948120117, 5.575007915496826, 5.55221..."
4,15,"[2184270, 1511610, 1262920, 1304290, 1263100]","[110800, 221680, 233840, 271590, 292730, 40971...","[8870, 12210, 22380, 47920, 55100, 206420, 219...","[4.440424919128418, 4.343510150909424, 4.32575..."


In [ ]:
user_eval['itemIndex_filtered'] = user_eval.apply(lambda x:[b for (b,z) in zip(x.steam_appid, x.rating) if b not in x.train_items], axis=1)
user_eval['rating_filtered'] = user_eval.apply(lambda x:[z for (b,z) in zip(x.steam_appid, x.rating) if b not in x.train_items], axis=1)

In [ ]:
user_eval.head()

,steam_int_id,steam_appid,ground_truth_items,train_items,rating,itemIndex_filtered,rating_filtered
0,3,"[2184270, 1511610, 1262920, 1304290, 1263100]",[410820],"[261570, 296490]","[4.871675491333008, 4.765350818634033, 4.74587...","[2184270, 1511610, 1262920, 1304290, 1263100]","[4.871675491333008, 4.765350818634033, 4.74587..."
1,6,"[2184270, 1511610, 1262920, 1304290, 1263100]",[1929610],"[739630, 771300, 962130, 1304930, 1677740]","[6.077251434326172, 5.944612979888916, 5.92031...","[2184270, 1511610, 1262920, 1304290, 1263100]","[6.077251434326172, 5.944612979888916, 5.92031..."
2,9,"[2184270, 1511610, 1262920, 1304290, 1263100]","[264710, 346110, 794260]","[251570, 354680, 424370, 455120, 728240, 79074...","[5.162646770477295, 5.0499701499938965, 5.0293...","[2184270, 1511610, 1262920, 1304290, 1263100]","[5.162646770477295, 5.0499701499938965, 5.0293..."
3,12,"[2184270, 1511610, 1262920, 1304290, 1263100]",[755790],"[221100, 267600, 506540, 677620, 1149460, 1307...","[5.699399948120117, 5.575007915496826, 5.55221...","[2184270, 1511610, 1262920, 1304290, 1263100]","[5.699399948120117, 5.575007915496826, 5.55221..."
4,15,"[2184270, 1511610, 1262920, 1304290, 1263100]","[110800, 221680, 233840, 271590, 292730, 40971...","[8870, 12210, 22380, 47920, 55100, 206420, 219...","[4.440424919128418, 4.343510150909424, 4.32575...","[2184270, 1511610, 1262920, 1304290, 1263100]","[4.440424919128418, 4.343510150909424, 4.32575..."


In [ ]:
import numpy as np
import math

def score(predicted, actual, metric, k=None):
    """
    Parameters
    ----------
    predicted : List
        List of predicted apps.
    actual : List
        List of masked apps.
    metric : 'precision', 'recall', or 'ndcg'
        A valid metric for recommendation.
    k : int or None, optional
        The value of k for precision and recall calculations. Default is None.
    Raises
    -----
    Returns
    -------
    m : float
        score.
    """
    valid_metrics = ['precision', 'recall', 'ndcg']
    if metric not in valid_metrics:
        raise Exception(f"Choose one valid baseline in the list: {valid_metrics}")

    if metric == 'precision':
        if k is None:
          m = np.mean([float(len(set(predicted[:k]) & set(actual))) / float(k)
                     for k in range(1, len(predicted) + 1)])
        else:
          m = np.mean([float(len(set(predicted[:k]) & set(actual))) / float(k)])
    elif metric == 'recall':
        if k is None:
            raise ValueError("Parameter 'k' must be specified for recall calculation.")
        m = np.mean([float(len(set(predicted[:k]) & set(actual))) / float(len(actual))])
    elif metric == 'ndcg':
        v = [1 if i in actual else 0 for i in predicted]
        v_2 = [1 for i in actual]
        dcg = sum([(2**i-1)/math.log(k+2, 2) for (k, i) in enumerate(v)])
        idcg = sum([(2**i-1)/math.log(k+2, 2) for (k, i) in enumerate(v_2)])
        m = dcg/idcg

    return m


In [ ]:
user_eval['precision'] = user_eval.apply(lambda x: score(x.itemIndex_filtered, x.ground_truth_items, 'precision'), axis=1)
user_eval['NDCG'] = user_eval.apply(lambda x: score(x.itemIndex_filtered, x.ground_truth_items, 'ndcg'), axis=1)

In [ ]:
user_eval['recall'] = user_eval.apply(lambda x: score(x.itemIndex_filtered, x.ground_truth_items, 'recall', 5), axis=1)
user_eval['precisionAt'] = user_eval.apply(lambda x: score(x.itemIndex_filtered, x.ground_truth_items, 'precision', 5), axis=1)

In [ ]:
user_eval.head()

In [ ]:
MAP = user_eval.precision.mean()
avg_NDCG = user_eval.NDCG.mean()
recall = user_eval.recall.mean()
precision = user_eval.precisionAt.mean()

In [ ]:
MAP

2.97416998027996e-06

In [ ]:
avg_NDCG

4.955919648603833e-06

In [ ]:
recall

4.892486526178699e-06

In [ ]:
precision

1.9396760740956263e-06

# Prova

In [ ]:
user_recs = als_model.recommendForUserSubset(user, 5)

In [ ]:
user_recs.show(truncate=False)

+------------+---------------+
|steam_int_id|recommendations|
+------------+---------------+
+------------+---------------+

